In [ ]:
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid


dataset = Planetoid(root='/content/Cora', name='Cora')
data = dataset[0]


class SimpleGCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 4)
        self.conv2 = GCNConv(4, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


model = SimpleGCN(
    in_channels=dataset.num_node_features,
    out_channels=dataset.num_classes
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch:02d}, Loss: {loss.item():.4f}")


model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()

print(f"\n GCN Test Accuracy: {acc:.4f}")


Processing...
Done!


Epoch 00, Loss: 1.9536
Epoch 10, Loss: 1.5395
Epoch 20, Loss: 1.0920
Epoch 30, Loss: 0.7307
Epoch 40, Loss: 0.4854

 GCN Test Accuracy: 0.6190
